### 1. Lectura de los datos

In [1]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\eliza\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# Cargar datos
data = pd.read_csv('C:/Users/eliza/OneDrive/Desktop/Claus/Konrad/NLP/reviews_vidjew_es.csv')
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


### 2. Pre-procesamiento

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto

data['Pre-Processed'] = data['review_body'].apply(lambda texto: pre_procesado(texto))
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,Pre-Processed
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


### 3. Modelo de LDA

In [5]:
#Representación de los documentos en forma de diccionario
dictionary = Dictionary(data['Pre-Processed'].values)

# Filtrar palabras muy frecuentes o poco frecuentes
dictionary.filter_extremes(no_below=5, no_above=0.5) 

# Corpus
corpus = [dictionary.doc2bow(text) for text in data['Pre-Processed'].values]

In [9]:
#Entrenamiento del modelo
model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 6, passes=50)

In [10]:
#Resultados
model.print_topics(num_words=10)

[(0,
  '0.058*"juego" + 0.038*"llegado" + 0.036*"día" + 0.023*"pedido" + 0.021*"regalo" + 0.017*"amazon" + 0.015*"sido" + 0.014*"recibido" + 0.014*"entregado" + 0.012*"llegar"'),
 (1,
  '0.036*"calidad" + 0.034*"mas" + 0.033*"juego" + 0.022*"esperaba" + 0.022*"demasiado" + 0.016*"mejor" + 0.016*"pequeña" + 0.015*"mala" + 0.014*"precio" + 0.013*"si"'),
 (2,
  '0.075*"calidad" + 0.048*"bien" + 0.044*"buena" + 0.036*"precio" + 0.024*"queda" + 0.022*"bonito" + 0.019*"bastante" + 0.015*"regalo" + 0.013*"puede" + 0.013*"gustado"'),
 (3,
  '0.044*"producto" + 0.035*"perfecto" + 0.028*"precio" + 0.028*"tiempo" + 0.025*"bien" + 0.021*"buen" + 0.019*"original" + 0.019*"cumple" + 0.017*"llegó" + 0.017*"entrega"'),
 (4,
  '0.041*"si" + 0.026*"bien" + 0.025*"producto" + 0.022*"mando" + 0.021*"caja" + 0.015*"solo" + 0.014*"llego" + 0.013*"ps" + 0.013*"puede" + 0.013*"funciona"'),
 (5,
  '0.035*"bien" + 0.027*"bonitos" + 0.026*"foto" + 0.025*"tamaño" + 0.022*"plata" + 0.018*"pulsera" + 0.018*"mal" + 

### 4. Visualización de LDA

In [11]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

### 5. Interpretación

- El modelo de clasificación de textos tiene un alto nivel de precisión, una variedad importante de métricas de evaluación que permiten conocer la sensibilidad del modelo de acuerdo a los requerimientos del investigador y el tipo de datos, y se adapta de manera sencilla a distintos formatos de información (texto, imágenes o números) teniendo en cuenta las etiquetas que se le asignen durante el ejercicio. No obstante, el etiquetado requiere un alto nivel de rigurosidad para que el modelo tengo un nivel óptimo de presición.

- El modelo de agrupación de textos utilizando el método del codo puede presentar confusión a la hora de determinar el kmeans, pero una vez resuelto este impase, agrupa de manera muy precisa los textos cortos y permite ajustar los pasos anteriores para que el investigador pueda determinar cuáles son las etiquetas más apropiadas para la clasificación de textos.

- El modelo de temas (LDA) no es el más óptimo para analizar textos cortos como opiniones de los clientes sobre un producto, porque puede clasificar palabras en 2 o más clusters sin su contexto, por ejemplo, clasifica "producto" y "material" en varios clusters, sin embargo en uno de los comentarios se hace referencia a que "el material del producto es malo", mientras que en otro el cliente señala que "el producto es de buen material".